#Linear Optimization using Simplex computation
Our objective here is minimize risk, while targeting a specific return

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
file = "All_Stocks.xlsx"

In [ ]:
data = pd.read_excel(file)

In [ ]:
data.head()

,Date,GOOG,AAPL,SBUX,TSLA
0,2019-04-01,1188.479980,49.138439,74.778992,47.737999
1,2019-05-01,1103.630005,42.869724,73.219505,37.032001
2,2019-06-01,1080.910034,48.651680,81.073700,44.692001
3,2019-07-01,1216.680054,52.368397,91.576622,48.321999
4,2019-08-01,1188.099976,51.311405,93.385139,45.122002


In [ ]:
data["Date"] = pd.to_datetime(data["Date"], format="%d-%b-%y")

In [ ]:
data.head()

,Date,GOOG,AAPL,SBUX,TSLA
0,2019-04-01,1188.479980,49.138439,74.778992,47.737999
1,2019-05-01,1103.630005,42.869724,73.219505,37.032001
2,2019-06-01,1080.910034,48.651680,81.073700,44.692001
3,2019-07-01,1216.680054,52.368397,91.576622,48.321999
4,2019-08-01,1188.099976,51.311405,93.385139,45.122002


In [ ]:
data.dtypes

Date    datetime64[ns]
GOOG           float64
AAPL           float64
SBUX           float64
TSLA           float64
dtype: object

Now we must calculate returns from base/start price

We only want to do that for int or float valued columns


In [ ]:
returns = data[[key for key in dict(data.dtypes) if dict(data.dtypes)[key] in ["int64", "float64"]]].pct_change()

In [ ]:
returns.head()

,GOOG,AAPL,SBUX,TSLA
0,NaN,NaN,NaN,NaN
1,-0.071394,-0.127573,-0.020855,-0.224266
2,-0.020587,0.134873,0.107269,0.206848
3,0.125607,0.076394,0.129548,0.081223
4,-0.023490,-0.020184,0.019749,-0.066222


In [ ]:
returns = returns[1:]

In [ ]:
stock_names = ["GOOG", "AAPL", "SBUX", "TSLA"]
stock_names

['GOOG', 'AAPL', 'SBUX', 'TSLA']

In [ ]:
stock_returns = returns[stock_names]

In [ ]:
stock_returns.head()

,GOOG,AAPL,SBUX,TSLA
1,-0.071394,-0.127573,-0.020855,-0.224266
2,-0.020587,0.134873,0.107269,0.206848
3,0.125607,0.076394,0.129548,0.081223
4,-0.023490,-0.020184,0.019749,-0.066222
5,0.026008,0.077038,-0.080829,0.067639


In [ ]:
n = len(stock_names)

In [ ]:
!pip install CVXOPT

In [ ]:
import cvxopt as opt

In [ ]:
from cvxopt import matrix, solvers

In [ ]:
expected_returns = np.mean(stock_returns)

In [ ]:
expected_returns

GOOG    0.026997
AAPL    0.045056
SBUX    0.019943
TSLA    0.148274
dtype: float64

In [ ]:
expected_returns = matrix(expected_returns)
expected_returns

<4x1 matrix, tc='d'>

In [ ]:
max_loss = np.min(stock_returns)
max_loss

GOOG   -0.131797
AAPL   -0.127573
SBUX   -0.157894
TSLA   -0.224266
dtype: float64

In [ ]:
max_loss = -max_loss
max_loss

GOOG    0.131797
AAPL    0.127573
SBUX    0.157894
TSLA    0.224266
dtype: float64

We define our objective as max loss <br>
So, our objective essentially is to minimize portfolio risk

In [ ]:
objective = matrix(max_loss)

Constraint: To ensure weights for all securities sum up to 100%, or 1.0

The LHS defines the coefficients for weights, for eg: </br>
a1*w1 + a2*w2 + a3*w3 = 100% (or 1.0) *italicized text* </br>
So, lhs essentially tracks A = [a1, a2, a3] <br>
RHS is the final pf weight, which should always be 1.0

In [ ]:
constraints_equality_lhs = matrix(1.0, (1,n))
constraints_equality_lhs

<1x4 matrix, tc='d'>

In [ ]:
constraints_equality_rhs = matrix(1.0)
constraints_equality_rhs

<1x1 matrix, tc='d'>

We define a constraint as: <br>
all weights must be >= 0, to ensure we don't get any negative weights<br>
This is done to disable short-selling

In [ ]:
constraints_inequality_lhs_row1 = matrix(-np.identity(n))

In [ ]:
constraints_inequality_rhs_row1 = matrix(0.0, (n, 1))

In [ ]:
constraints_inequality_lhs_row1

<4x4 matrix, tc='d'>

In [ ]:
constraints_inequality_rhs_row1

<4x1 matrix, tc='d'>

We add another constraint on beating the expected returns

In [ ]:
constraints_inequality_lhs_row2 = matrix(-np.transpose(np.array(expected_returns)))
constraints_inequality_lhs_row2

<1x4 matrix, tc='d'>

In [ ]:
min_expected_return = 0.02

In [ ]:
constraints_inequality_rhs_row2 = matrix(-np.ones((1, 1)) * min_expected_return)

In [ ]:
constraints_inequality_rhs_row2

<1x1 matrix, tc='d'>

In [ ]:
constraints_inequality_lhs_combined = matrix(np.concatenate((constraints_inequality_lhs_row1, constraints_inequality_lhs_row2), 0))

In [ ]:
constraints_inequality_lhs_combined

<5x4 matrix, tc='d'>

In [ ]:
constraints_inequality_rhs_combined = matrix(np.concatenate((constraints_inequality_rhs_row1, constraints_inequality_rhs_row2), 0))

In [ ]:
constraints_inequality_rhs_combined

<5x1 matrix, tc='d'>

In [ ]:
solution = solvers.lp(objective, constraints_inequality_lhs_combined, constraints_inequality_rhs_combined, constraints_equality_lhs, constraints_equality_rhs)

     pcost       dcost       gap    pres   dres   k/t
 0:  1.6010e-01  1.8075e-01  1e+00  3e-18  2e+00  1e+00
 1:  1.5759e-01  1.5766e-01  4e-02  2e-16  9e-02  4e-02
 2:  1.2913e-01  1.2649e-01  9e-03  4e-16  1e-02  3e-03
 3:  1.2817e-01  1.2796e-01  7e-04  1e-16  1e-03  2e-04
 4:  1.2758e-01  1.2758e-01  1e-05  2e-16  1e-05  3e-06
 5:  1.2757e-01  1.2757e-01  1e-07  2e-16  1e-07  3e-08
 6:  1.2757e-01  1.2757e-01  1e-09  2e-16  1e-09  3e-10
Optimal solution found.


In [ ]:
weights_linear = solution["x"]
weights_linear

<4x1 matrix, tc='d'>

In [ ]:
weights_linear = np.array(weights_linear)

In [ ]:
weights_linear

array([[1.32313735e-07],
       [9.99999865e-01],
       [2.05097927e-09],
       [5.88424969e-10]])

In [ ]:
sum(weights_linear)

array([1.])

In [ ]:
pf_risk_linear = np.dot(weights_linear.T, max_loss)

In [ ]:
pf_risk_linear

array([0.12757253])

In [ ]:
pf_return_linear = np.dot(expected_returns.T, weights_linear)

In [ ]:
pf_return_linear

array([[0.04505594]])